In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Time
import time

# Random and os for reproducibility
import random
import os

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
import tensorflow as tf

# Processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Pipeline
from sklearn.pipeline import Pipeline

# Data imputation
from sklearn.impute import SimpleImputer

# Making keras compatible with scikit learn api
# https://scikit-learn.org/stable/developers/develop.html
from sklearn.base import BaseEstimator, RegressorMixin

# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

# Converting Times Series Data to 3D format
from src.utils.split_sequences import split_sequences

# To run cross validation parallelized
from joblib import Parallel, delayed

2024-02-15 10:30:55.897218: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-15 10:30:55.899452: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-15 10:30:55.946304: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-15 10:30:55.947352: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-02-15 10:30:56.728198: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


<IPython.core.display.Javascript object>

# Functions and definitions

## Helper functions for blocked time series cross validation

### Convert train/test data to 3D format

In [3]:
def generate_sequences_helper(
    dataset, cement_types, dates=None, timesteps=None, split_by_cement_type=False
):
    index_train = dataset["y_train"].index
    index_test = dataset["y_test"].index

    dataset["y_train"] = dataset["y_train"].reset_index(drop=True)
    dataset["y_test"] = dataset["y_test"].reset_index(drop=True)

    if dates is not None:
        dataset["dates_train"] = dates[index_train].reset_index(drop=True)
        dataset["dates_test"] = dates[index_test].reset_index(drop=True)

    dataset["cement_types_train"] = cement_types.loc[index_train].reset_index(drop=True)
    dataset["cement_types_test"] = cement_types.loc[index_test].reset_index(drop=True)

    dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

    return dataset

<IPython.core.display.Javascript object>

In [4]:
def generate_sequences(dataset, timesteps, split_by_cement_type=False):
    if split_by_cement_type:
        dataset["x_train"], dataset["y_train"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_train"],
                    pd.DataFrame(dataset["x_train"], columns=x.columns),
                    dataset["cement_types_train"],
                    dataset["y_train"],
                ],
                axis=1,
            ),
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_test"],
                    pd.DataFrame(dataset["x_test"], columns=x.columns),
                    dataset["cement_types_test"],
                    dataset["y_test"],
                ],
                axis=1,
            ),
            timesteps,
        )
    else:
        dataset["x_train"], dataset["y_train"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_train"], columns=x.columns),
                    dataset["y_train"],
                ],
                axis=1,
            ).values,
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_test"], columns=x.columns),
                    dataset["y_test"],
                ],
                axis=1,
            ).values,
            timesteps,
        )
    return dataset

<IPython.core.display.Javascript object>

### Data preprocessing

In [5]:
def impute_data(dataset, imputer=None, imputer_params=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply imputation to the data
    if imputer is not None:
        imputer = imputer() if imputer_params is None else imputer(**imputer_params)
        x_train = imputer.fit_transform(x_train)
        x_test = imputer.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [6]:
def transform_data(dataset, transformer=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply data normalization/standardization to the data
    if transformer is not None:
        scaler = transformer()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [7]:
def preprocess_data(dataset, transformer=None, imputer=None, imputer_params=None):
    dataset = impute_data(dataset, imputer, imputer_params)
    dataset = transform_data(dataset, transformer)
    return dataset

<IPython.core.display.Javascript object>

### Train and evaluate the model

In [8]:
def train_and_evaluate_model(Estimator, dataset, estimator_params=None):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Instantiate the model
    model = Estimator() if estimator_params is None else Estimator(**estimator_params)

    # Fitting the model
    model.fit(x_train, y_train)

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

### Custom Cross Validate

In [9]:
def custom_cross_validate_parallelized(
    Estimator,
    Imputer,
    Transform,
    x,
    y,
    cv,
    timesteps,
    dates=None,
    cement_types=None,
    estimator_params=None,
    imputer_params=None,
    split_by_cement_type=True,
    n_jobs=-1,  # Set the number of parallel jobs, -1 for using all available cores
):
    def process_fold(train_index, test_index, dates, cement_types, x, y):
        dataset = {
            "dates_train": dates[train_index].reset_index(drop=True),
            "cement_types_train": cement_types.loc[train_index].reset_index(drop=True),
            "x_train": x.loc[train_index].reset_index(drop=True),
            "y_train": y[train_index].reset_index(drop=True),
            "dates_test": dates[test_index].reset_index(drop=True),
            "cement_types_test": cement_types.loc[test_index].reset_index(drop=True),
            "x_test": x.loc[test_index].reset_index(drop=True),
            "y_test": y[test_index].reset_index(drop=True),
        }

        set_seeds()

        # Preprocess the dataset
        dataset = preprocess_data(dataset, Transform, Imputer, imputer_params)

        # generate sequences (3D format)
        dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

        # Train and Evaluate the model
        score = train_and_evaluate_model(Estimator, dataset, estimator_params)
        return score

    scores = Parallel(n_jobs=n_jobs)(
        delayed(process_fold)(train_index, test_index, dates, cement_types, x, y)
        for train_index, test_index in cv.split(x)
    )

    # After every iteration metrics results are appended together
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += [value]

    results = [scores_final]
    return results

<IPython.core.display.Javascript object>

## Model Definition

In [10]:
class BidirectionalLSTM(BaseEstimator, RegressorMixin):
    def __init__(self):
        self.model = self.get_model()
        self.batch_size = 32
        self.epochs = 300
        self.verbose = 0

    def fit(self, X=None, y=None):
        self.model.fit(
            X, y, batch_size=self.batch_size, epochs=self.epochs, verbose=self.verbose
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=32, activation="relu")
            )
        ),
        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

In [11]:
def pad_time_series(dataframe, timesteps):
    """
    Pad timeseries with zeros
    """
    df_tmp = pd.DataFrame(
        dict(
            zip(
                dataframe.columns,
                [[0 for _ in range(timesteps - 1)] for _ in range(dataframe.shape[1])],
            )
        )
    )
    df_tmp[DATE] = dataframe[DATE].iloc[0]
    return pd.concat([df_tmp, dataframe], axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

In [12]:
def split_sequences_per_cement_type(dataframe, timesteps, pad=False):
    """
    Create sequences per cement time
    to avoid having parts of the sequence
    of different types of cement.
    """
    if timesteps == 1:
        return split_sequences(
            dataframe.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps
        )

    dates = dataframe[DATE][timesteps - 1 :]
    data = []
    dataframes = []

    for cement_type in CEMENT_TYPES:
        data.append(dataframe[dataframe[cement_type] == 1])
    data.append(dataframe[(dataframe[CEMENT_TYPES] == 0).all(axis=1)])

    for df in data:
        if pad:
            dates = df[DATE].reset_index(drop=True)
            df = pad_time_series(df, timesteps).reset_index(drop=True)
        else:
            dates = df[DATE][timesteps - 1 :].reset_index(drop=True)
        x, y = split_sequences(df.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps)
        x = pd.DataFrame({"Sequences": [sample.tolist() for sample in x]})
        y = pd.DataFrame({"Target": y})
        dataframes.append(pd.concat([dates, x, y], axis=1))

    data = pd.concat(dataframes, axis=0)
    data[DATE] = pd.to_datetime(data[DATE])
    data = data.sort_values(by=DATE).reset_index(drop=True)
    x = data["Sequences"]
    y = data["Target"].values
    x = np.array(x.tolist())

    return x, y

<IPython.core.display.Javascript object>

# Settings for Reproducibility

In [13]:
SEED = 47

<IPython.core.display.Javascript object>

In [14]:
def set_seeds(seed=SEED):
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)


# https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed

<IPython.core.display.Javascript object>

In [15]:
def set_global_determinism():
    set_seeds(seed=SEED)

    os.environ["TF_DETERMINISTIC_OPS"] = "1"
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

<IPython.core.display.Javascript object>

In [16]:
def suppress_warnings():
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

<IPython.core.display.Javascript object>

In [17]:
def supress_other_stuff():
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    # os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

<IPython.core.display.Javascript object>

In [18]:
suppress_warnings()
supress_other_stuff()

<IPython.core.display.Javascript object>

In [19]:
index_to_save = 9

<IPython.core.display.Javascript object>

In [20]:
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}
DATE = "Date"
CEMENT_TYPES = ["Cement_Type_CP II-F-40"]

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [21]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "209",
    "Plant": "AN",
    "Features": "Chemical + Feature Engineering",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "BidirectionalLSTM",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [22]:
df = pd.read_csv("../../../../../data/processed/209/an.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [23]:
df_copy = df.copy()
df_copy = pd.get_dummies(data=df_copy, columns=["Cement_Type"], drop_first=True)

df_copy = df_copy.drop(
    [
        "Blaine",
        "#200",
        "#325",
        "Final setting time",
"Initial setting time",
        
        
        "CS3",
        "CS7",
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

## Feature Engineering

In [24]:
# Feature Engineering over Chemical Features
# Feature Engineering over Chemical Features
ch_features = [
    "CaO",
    "MgO",
    "Na2O",
    "Al2O3",
    "SiO2",
    "SO3",
    "K2O",
    "Fe2O3",
]

df_copy["std_ch_feats"] = df_copy[ch_features].std(ddof=0, axis=1)

df_copy["std_ch_feats"] = df_copy[ch_features].std(ddof=0, axis=1)

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we use all available features

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [25]:
df_copy[CEMENT_TYPES] = df_copy[CEMENT_TYPES].astype(int)
dates = df["Date"].copy()
y = df_copy.pop("CS28")
x = df_copy
df_copy = pd.concat([x, y], axis=1)

<IPython.core.display.Javascript object>

In [26]:
TIMESTEPS_LIST = [1, 7, 14]

<IPython.core.display.Javascript object>

# 1. Long Short Term Memory - BidirectionalLSTM

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Timesteps:</b> 1, 3, 5, 7, 10, 15, 20<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [27]:
start = time.time()

repeats = 3
n_splits = 5
TIMESTEPS_LIST = [1, 7, 14]

print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
    x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
    y = df_copy["CS28"]
    scores = custom_cross_validate_parallelized(
        BidirectionalLSTM,
        SimpleImputer,
        StandardScaler,
        x,
        y,
        cv,
        timesteps=timesteps,
        dates=dates,
        cement_types=df_copy[CEMENT_TYPES],
        estimator_params=None,
        imputer_params={"strategy": "median"},
        split_by_cement_type=True,
    )
    scores = scores[0]
    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores, METRICS, METRICS_DICT)

    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Repeated KFold"
    results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Repeated Cross Validation:
Repeats: 3
n_splits: 5



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.579 (0.040)
MAE: 1.201 (0.026)
MAPE: 0.030 (0.001)
R2: 0.903 (0.005)


******
[TEST]
******
RMSE: 1.923 (0.244)
MAE: 1.402 (0.112)
MAPE: 0.034 (0.003)
R2: 0.853 (0.035)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.158 (0.123)
MAE: 0.911 (0.102)
MAPE: 0.023 (0.003)
R2: 0.947 (0.011)


******
[TEST]
******
RMSE: 1.867 (0.492)
MAE: 1.366 (0.159)
MAPE: 0.034 (0.004)
R2: 0.857 (0.079)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.037 (0.057)
MAE: 0.823 (0.057)
MAPE: 0.020 (0.001)
R2: 0.958 (0.005)


******
[TEST]
******
RMSE: 1.859 (0.622)
MAE: 1.428 (0.259)
MAPE: 0.035 (0.006)
R2: 0.853 (0.128)


Minutes Elapsed:  4.6828932086626684


<IPython.core.display.Javascript object>

In [28]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model Cross Validation  \
                                                                        
0  Chemical + Feature Engineering  BidirectionalLSTM   Repeated KFold   
1  Chemical + Feature Engineering  BidirectionalLSTM   Repeated KFold   
2  Chemical + Feature Engineering  BidirectionalLSTM   Repeated KFold   

  Timesteps RMSE Test            MAE Test           MAPE Test            \
                 mean       std      mean       std      mean       std   
0         1  1.922820  0.243587  1.401867  0.112309  0.034469  0.002673   
1         7  1.866598  0.492288  1.365564  0.159411  0.033919  0.003784   
2        14  1.859021  0.622327  1.428369  0.259079  0.035241  0.006107   

    R2 Test            
       mean       std  
0  0.853100  0.035132  
1  0.856964  0.078660  
2  0.852872  0.128054

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [29]:
start = time.time()

repeats = 3
n_splits = 5
train_size = 0.8
TIMESTEPS_LIST = [1, 7, 14]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
        y = df_copy["CS28"]

        cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)

        scores = custom_cross_validate_parallelized(
            BidirectionalLSTM,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            cement_types=df_copy[CEMENT_TYPES],
            estimator_params=None,
            imputer_params={"strategy": "median"},
            split_by_cement_type=True,
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "train_size": 0.8}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 5.200 (0.544)
MAE: 3.902 (0.343)
MAPE: 0.099 (0.010)
R2: -0.142 (0.339)


******
[TEST]
******
RMSE: 7.866 (1.112)
MAE: 5.636 (1.121)
MAPE: 0.143 (0.028)
R2: -1.555 (0.907)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.309 (0.214)
MAE: 1.036 (0.194)
MAPE: 0.025 (0.005)
R2: 0.927 (0.032)


******
[TEST]
******
RMSE: 12.070 (9.056)
MAE: 10.036 (8.319)
MAPE: 0.254 (0.206)
R2: -7.417 (10.358)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.379 (0.313)
MAE: 1.150 (0.283)
MAPE: 0.029 (0.007)
R2: 0.922 (0.030)


******
[TEST]
******
RMSE: 8.644 (6.109)
MAE: 7.404 (5.502)
MAPE: 0.183 (0.135)
R2: -7.253 (9.728)


Minutes Elapsed:  1.6991418123245239


<IPython.core.display.Javascript object>

In [30]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                       
0  Chemical + Feature Engineering  BidirectionalLSTM   
1  Chemical + Feature Engineering  BidirectionalLSTM   
2  Chemical + Feature Engineering  BidirectionalLSTM   
3  Chemical + Feature Engineering  BidirectionalLSTM   
4  Chemical + Feature Engineering  BidirectionalLSTM   
5  Chemical + Feature Engineering  BidirectionalLSTM   

             Cross Validation Timesteps  RMSE Test             MAE Test  \
                                              mean       std       mean   
0  Blocking Time Series Split         1   7.865860  1.111813   5.636153   
1  Blocking Time Series Split         7  12.069509  9.056446  10.036392   
2  Blocking Time Series Split        14   8.643809  6.109032   7.403586   
3              Repeated KFold         1   1.922820  0.243587   1.401867   
4              Repeated KFold         7   1.866598  0.492288   1.365564   
5              Repeated KFold        14   1.859021  0.622327   1.428369   

            MAPE Test             R2 Test             
        std      mean       std      mean        std  
0  1.121310  0.143413  0.027675 -1.554890   0.906903  
1  8.319490  0.253983  0.205915 -7.417388  10.357521  
2  5.502263  0.183230  0.135357 -7.253264   9.727747  
3  0.112309  0.034469  0.002673  0.853100   0.035132  
4  0.159411  0.033919  0.003784  0.856964   0.078660  
5  0.259079  0.035241  0.006107  0.852872   0.128054

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [31]:
set_seeds()
start = time.time()
gap = 0
n_splits = 5
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
        y = df_copy["CS28"]

        cv = TimeSeriesSplit(
            gap=gap, max_train_size=None, n_splits=n_splits, test_size=None
        )
        scores = custom_cross_validate_parallelized(
            BidirectionalLSTM,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            cement_types=df_copy[CEMENT_TYPES],
            estimator_params=None,
            imputer_params={"strategy": "median"},
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "Gap": 0}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 2.363 (1.200)
MAE: 1.739 (0.860)
MAPE: 0.044 (0.023)
R2: 0.689 (0.334)


******
[TEST]
******
RMSE: 4.256 (2.688)
MAE: 2.893 (1.564)
MAPE: 0.072 (0.040)
R2: -0.151 (1.225)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.096 (0.203)
MAE: 0.875 (0.155)
MAPE: 0.022 (0.003)
R2: 0.947 (0.018)


******
[TEST]
******
RMSE: 4.491 (2.858)
MAE: 3.358 (2.024)
MAPE: 0.083 (0.051)
R2: -0.329 (1.462)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.149 (0.259)
MAE: 0.969 (0.251)
MAPE: 0.024 (0.006)
R2: 0.940 (0.029)


******
[TEST]
******
RMSE: 3.932 (1.995)
MAE: 2.982 (1.445)
MAPE: 0.075 (0.037)
R2: 0.107 (0.883)


Minutes Elapsed:  6.8134119232495625


<IPython.core.display.Javascript object>

In [32]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                       
0  Chemical + Feature Engineering  BidirectionalLSTM   
1  Chemical + Feature Engineering  BidirectionalLSTM   
2  Chemical + Feature Engineering  BidirectionalLSTM   
3  Chemical + Feature Engineering  BidirectionalLSTM   
4  Chemical + Feature Engineering  BidirectionalLSTM   
5  Chemical + Feature Engineering  BidirectionalLSTM   
6  Chemical + Feature Engineering  BidirectionalLSTM   
7  Chemical + Feature Engineering  BidirectionalLSTM   
8  Chemical + Feature Engineering  BidirectionalLSTM   

             Cross Validation Timesteps  RMSE Test             MAE Test  \
                                              mean       std       mean   
0  Blocking Time Series Split         1   7.865860  1.111813   5.636153   
1  Blocking Time Series Split         7  12.069509  9.056446  10.036392   
2  Blocking Time Series Split        14   8.643809  6.109032   7.403586   
3              Repeated KFold         1   1.922820  0.243587   1.401867   
4              Repeated KFold         7   1.866598  0.492288   1.365564   
5              Repeated KFold        14   1.859021  0.622327   1.428369   
6           Time Series Split         1   4.255577  2.687787   2.892927   
7           Time Series Split         7   4.490843  2.857809   3.357673   
8           Time Series Split        14   3.932365  1.995058   2.981558   

            MAPE Test             R2 Test             
        std      mean       std      mean        std  
0  1.121310  0.143413  0.027675 -1.554890   0.906903  
1  8.319490  0.253983  0.205915 -7.417388  10.357521  
2  5.502263  0.183230  0.135357 -7.253264   9.727747  
3  0.112309  0.034469  0.002673  0.853100   0.035132  
4  0.159411  0.033919  0.003784  0.856964   0.078660  
5  0.259079  0.035241  0.006107  0.852872   0.128054  
6  1.564343  0.071793  0.040083 -0.151228   1.224668  
7  2.023757  0.083164  0.051286 -0.328632   1.462175  
8  1.445489  0.075443  0.036520  0.106657   0.883341

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [33]:
start = time.time()
test_size = 0.2
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Out of time Cross Val:")
print(f"Repeats: {repeats}")
print(f"Train: {80}%", f"Test: {20}%")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()

    def parallel_repeats(repeat):
        set_seeds(SEED + repeat)

        scores_final = None

        # Data Splitting
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1).copy()
        y = df_copy["CS28"].copy()

        x_train, x_test, y_train, y_test = train_test_split(
            x, y, test_size=test_size, random_state=SEED, shuffle=False
        )

        # Preprocessing
        imputer = SimpleImputer(strategy="median")
        scaler = StandardScaler()

        x_train = imputer.fit_transform(x_train)
        x_train = scaler.fit_transform(x_train)
        dates_train = dates[: x_train.shape[0]].reset_index(drop=True)
        cement_types_train = df_copy[CEMENT_TYPES][: x_train.shape[0]].reset_index(
            drop=True
        )

        x_test = imputer.transform(x_test)
        x_test = scaler.transform(x_test)
        dates_test = dates[x_train.shape[0] :].reset_index(drop=True)
        cement_types_test = df_copy[CEMENT_TYPES][x_train.shape[0] :].reset_index(
            drop=True
        )

        # Sequence Splitting
        data_train = pd.concat(
            [
                dates_train,
                pd.DataFrame(x_train, columns=x.columns),
                cement_types_train,
                y_train.reset_index(drop=True),
            ],
            axis=1,
        )
        data_test = pd.concat(
            [
                dates_test,
                pd.DataFrame(x_test, columns=x.columns),
                cement_types_test,
                y_test.reset_index(drop=True),
            ],
            axis=1,
        )

        x_train, y_train = split_sequences_per_cement_type(data_train, timesteps)
        x_test, y_test = split_sequences_per_cement_type(data_test, timesteps)

        # Train model and test evalutation
        # Fit model
        pipeline = Pipeline([("estimator", BidirectionalLSTM())])
        pipeline.fit(x_train, y_train)

        # Make predictions
        y_train_pred = pipeline.predict(x_train)
        y_test_pred = pipeline.predict(x_test)

        # evaluate predictions
        scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

        return scores_final

    scores = Parallel(n_jobs=5)(
        delayed(parallel_repeats)(repeat) for repeat in range(repeats)
    )
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += value

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    # scores = {key: val[0] for key, val in scores.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Out of time Split"
    results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(
        results_dict_copy, {key: value for key, value in scores_final.items()}
    )
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Out of time Cross Val:
Repeats: 3
Train: 80% Test: 20%



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.564 (0.018)
MAE: 1.177 (0.017)
MAPE: 0.029 (0.000)
R2: 0.901 (0.002)


******
[TEST]
******
RMSE: 1.822 (0.014)
MAE: 1.412 (0.017)
MAPE: 0.034 (0.000)
R2: 0.882 (0.002)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 0.962 (0.060)
MAE: 0.764 (0.053)
MAPE: 0.019 (0.001)
R2: 0.963 (0.005)


******
[TEST]
******
RMSE: 1.720 (0.094)
MAE: 1.427 (0.066)
MAPE: 0.034 (0.002)
R2: 0.894 (0.011)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 0.929 (0.073)
MAE: 0.733 (0.067)
MAPE: 0.018 (0.002)
R2: 0.965 (0.006)


******
[TEST]
******
RMSE: 1.969 (0.186)
MAE: 1.562 (0.114)
MAPE: 0.037 (0.003)
R2: 0.860 (0.027)


Minutes Elapsed:  2.285976537068685


<IPython.core.display.Javascript object>

In [34]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)


Features              Model  \
                                                        
0   Chemical + Feature Engineering  BidirectionalLSTM   
1   Chemical + Feature Engineering  BidirectionalLSTM   
2   Chemical + Feature Engineering  BidirectionalLSTM   
3   Chemical + Feature Engineering  BidirectionalLSTM   
4   Chemical + Feature Engineering  BidirectionalLSTM   
5   Chemical + Feature Engineering  BidirectionalLSTM   
6   Chemical + Feature Engineering  BidirectionalLSTM   
7   Chemical + Feature Engineering  BidirectionalLSTM   
8   Chemical + Feature Engineering  BidirectionalLSTM   
9   Chemical + Feature Engineering  BidirectionalLSTM   
10  Chemical + Feature Engineering  BidirectionalLSTM   
11  Chemical + Feature Engineering  BidirectionalLSTM   

              Cross Validation Timesteps  RMSE Test             MAE Test  \
                                               mean       std       mean   
0   Blocking Time Series Split         1   7.865860  1.111813   5.636153   
1   Blocking Time Series Split         7  12.069509  9.056446  10.036392   
2   Blocking Time Series Split        14   8.643809  6.109032   7.403586   
3            Out of time Split         1   1.821766  0.014185   1.411929   
4            Out of time Split         7   1.719942  0.093845   1.427352   
5            Out of time Split        14   1.969055  0.186145   1.561864   
6               Repeated KFold         1   1.922820  0.243587   1.401867   
7               Repeated KFold         7   1.866598  0.492288   1.365564   
8               Repeated KFold        14   1.859021  0.622327   1.428369   
9            Time Series Split         1   4.255577  2.687787   2.892927   
10           Time Series Split         7   4.490843  2.857809   3.357673   
11           Time Series Split        14   3.932365  1.995058   2.981558   

             MAPE Test             R2 Test             
         std      mean       std      mean        std  
0   1.121310  0.143413  0.027675 -1.554890   0.906903  
1   8.319490  0.253983  0.205915 -7.417388  10.357521  
2   5.502263  0.183230  0.135357 -7.253264   9.727747  
3   0.017159  0.034073  0.000456  0.881996   0.001834  
4   0.066170  0.034447  0.001644  0.894405   0.011294  
5   0.113891  0.037433  0.002650  0.860488   0.026934  
6   0.112309  0.034469  0.002673  0.853100   0.035132  
7   0.159411  0.033919  0.003784  0.856964   0.078660  
8   0.259079  0.035241  0.006107  0.852872   0.128054  
9   1.564343  0.071793  0.040083 -0.151228   1.224668  
10  2.023757  0.083164  0.051286 -0.328632   1.462175  
11  1.445489  0.075443  0.036520  0.106657   0.883341

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [35]:
path = "../../../../../reports/results/local_models/209/an/full/"
filename = f"BidirectionalLSTM_results_full_{index_to_save}.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [36]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../reports/results/local_models/209/an/grouped/"
filename = f"BidirectionalLSTM_results_grouped_{index_to_save}.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>